In [1]:
# インポート
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

# データセットの用意
data_train = pd.read_csv('./input/train.csv')
data_train = data_train[['GrLivArea', 'YearBuilt', 'SalePrice']].astype('float')
#data_test = pd.read_csv('./input/test.csv')
#X_test_submission = data_test[['GrLivArea', 'YearBuilt']].values

# 標準化
scaler = StandardScaler()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

# 説明・目的変数
X = data_train[:, :2]
y = data_train[:, 2:].ravel()

# train.csvを学習用（train）8割、検証用（val）2割に分割する
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8, test_size=0.2)

# 【問題1】ブレンディング
ブレンディングとは? 以下分類だがわかりやすかった。  
<br>
1111100000  正解文字列（Ground truth）

1110100000  強学習器 (90%)  ← 良いモデルの予測結果は90%だった  
ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
<br>
1111100000  正解文字列（Ground truth）

1011110100  弱学習器 (70%) ：1をよく予測できる

1101000010  弱学習器　(70%) ：0をよく予測できる

0110101001  弱学習器  (60%) ：何もうまく予測できない

1111100000  弱学習器の投票平均 (100%)  ← 複数のビミョーなモデルたちの平均を取ってみると100%になった
  
これが一番単純なブレンディング  
とりあえずはこれを試してみる

In [2]:
# 各モデルインスタンス格納用
clf_dict = {}

# 線形回帰
clf_dict['LinearRegression'] = LinearRegression()

# SVM
clf_dict['SVR']  = SVR(gamma='auto')

# 決定木
clf_dict['DecisionTreeRegressor']  = DecisionTreeRegressor()

for clf_key in clf_dict:
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 
    
    # 予測実施
    y_pred = clf_dict[clf_key].predict(X_test)

    # mse
    print('{}のmse : {}'.format(clf_key, mean_squared_error(y_test, y_pred)))

LinearRegressionのmse : 0.317178319922263
SVRのmse : 0.33078207666781
DecisionTreeRegressorのmse : 0.5441879749035574


In [3]:
# 初期化
y_pred = np.zeros((len(X_test)))
w = 0.2

# blending
for clf_key in clf_dict:
    # 学習
    clf_dict[clf_key].fit(X_train, y_train) 

    # 予測実施
    y_pred_tmp = clf_dict[clf_key].predict(X_test)

    y_pred += y_pred_tmp * w
    
    w += 0.15

# mse
print('blendingごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

blendingごのmse : 0.36463370642137466


ちょうど中間くらい

# 【問題2】バギング
bootstrap aggregatingの略。
各学習器に使う学習用データをブートストラップサンプリングによって得て、  
その学習した学習器を予測に用いて最後アンサンブルするという方法。らしい  
ランダムフォレストとも言われる？ 
とりあえずやってみる

In [4]:
# 決定木
dtc = DecisionTreeRegressor()

# 初期化
y_pred = np.zeros((len(X_test)))
count = 0

for i in range(5):
    # ブートストラップサンプリング
    X_train, _, y_train, _ = train_test_split(X, y,train_size=0.8, test_size=0.2, shuffle=True)
    
    # 学習
    dtc.fit(X_train, y_train) 
    
    # 予測実施
    y_pred += dtc.predict(X_test)

    print('{}回目のmse : {}'.format(i+1, mean_squared_error(y_test, dtc.predict(X_test))))
    
    count += 1
    
y_pred = y_pred / count

# mse
print('bagginごのmse : {}'.format(mean_squared_error(y_test, y_pred)))

1回目のmse : 0.07750116494920367
2回目のmse : 0.07664014531503863
3回目のmse : 0.051393467598816765
4回目のmse : 0.09118663944392327
5回目のmse : 0.2521036434243572
bagginごのmse : 0.036127714364734324


これはおkそう　他のブレンディングとスタッキングを重点的にやる

# 【問題3】スタッキング
名前の通り訓練データを積み上げていく
<br>
複数学習機で推定→推定結果を特徴量として学習し推定→...を繰り返していくらしい
<br>
まずは二つのステージでやってみる K=4, M=3

In [29]:
# ステージ0
# 各モデルインスタンス格納用
clf_dict = {}

# 線形回帰
clf_dict['LinearRegression'] = LinearRegression()

# SVM
clf_dict['SVR']  = SVR(gamma='auto')

# 決定木
clf_dict['DecisionTreeRegressor']  = DecisionTreeRegressor()

# なん分割するか
K = 4
test_len = int(len(X)* 1 / K)
print(test_len)
kf = KFold(n_splits=K)


# 予測結果格納用(1/K = 検証データ)
new_feats = np.empty((0, test_len), int)

for clf_key in clf_dict:
    # 予測を初期化
    y_pred = np.zeros((test_len))
    for train_index, test_index in kf.split(X, y):
        # 学習
        clf_dict[clf_key].fit(X[train_index], y[train_index])

        # 予測結果を特徴量として格納
        y_pred += clf_dict[clf_key].predict(X[test_index])
    
    new_feats = np.append(new_feats, np.array([y_pred / K]), axis=0)


# ステージ1
pca = PCA(n_components = 2)
pca = pca.fit(new_feats.T)
new_feats = pca.transform(new_feats.T)

svr = DecisionTreeRegressor()
svr.fit(new_feats, y[test_index])

y_pred = svr.predict(new_feats)

# mse
print('stackingごのmse : {}'.format(mean_squared_error(y[test_index], y_pred)))

lr = LinearRegression()
lr.fit(new_feats, y[test_index])

y_pred = lr.predict(new_feats)

# mse
print('stackingごのmse : {}'.format(mean_squared_error(y[test_index], y_pred)))

365
stackingごのmse : 0.0
stackingごのmse : 0.7998896139423172


多分そもそもやり方が違いそう（＾－＾）
# 【問題4】（アドバンス課題）Microsoft Malware Prediction